<a href="https://colab.research.google.com/github/zhangou888/NN/blob/main/PyTorch_Neural_Network_on_Titanic_Dataset_(Binary_Classification)_with_SHAP_Explainability_and_Gradio_Interface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# PyTorch Neural Network on Titanic Dataset (Binary Classification) with SHAP Explainability and Gradio Interface

import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
import shap  # SHAP explainability
import gradio as gr  # Web UI
import seaborn as sns
import numpy as np

# Step 1: Load Titanic dataset
# You can use seaborn or load from a CSV if using Kaggle
data = sns.load_dataset('titanic')

# Step 2: Drop columns with many missing values or not useful for modeling
cols_to_drop = ['deck', 'embark_town', 'alive', 'class', 'who']
data = data.drop(columns=cols_to_drop)

# Step 3: Drop remaining rows with missing values to simplify processing
data = data.dropna()

# Step 4: Encode categorical columns ('sex' and 'embarked') into numeric
label_cols = ['sex', 'embarked']
for col in label_cols:
    data[col] = LabelEncoder().fit_transform(data[col])

# Step 5: Select input features and label
features = ['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked']
X = data[features].values  # Input features
y = data['survived'].values  # Target: 0 = did not survive, 1 = survived

# Step 6: Normalize features for stable training
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Step 7: Split dataset into train/test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 8: Convert NumPy arrays to PyTorch tensors for model training
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test_tensor  = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor  = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

# Wrap tensors in PyTorch DataLoader to manage batching during training
dataset_train = TensorDataset(X_train_tensor, y_train_tensor)
dataset_test  = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(dataset_train, batch_size=32, shuffle=True)
test_loader  = DataLoader(dataset_test, batch_size=32)

# Step 9: Define a simple feedforward neural network for binary classification
class TitanicNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(7, 16)  # Input: 7 features → 16 hidden neurons
        self.fc2 = nn.Linear(16, 8)  # Hidden layer → 8 neurons
        self.fc3 = nn.Linear(8, 1)   # Output layer → 1 neuron (binary output)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))  # Sigmoid for binary classification output
        return x

# Step 10: Initialize model, optimizer, and loss function
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TitanicNet().to(device)
criterion = nn.BCELoss()  # Binary cross-entropy loss
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Step 11: Train the model for a fixed number of epochs
epochs = 30
for epoch in range(epochs):
    model.train()
    total_loss = 0.0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss:.4f}")

# Step 12: Evaluate model on test set
model.eval()
preds = []
labels = []
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.to(device)
        outputs = model(X_batch)
        preds.extend(outputs.cpu().numpy())
        labels.extend(y_batch.numpy())

# Convert probabilities to binary predictions
preds_binary = [1 if p >= 0.5 else 0 for p in preds]
accuracy = accuracy_score(labels, preds_binary)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

# Step 13: Visualize confusion matrix
cm = confusion_matrix(labels, preds_binary)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Not Survived', 'Survived'])
disp.plot(cmap='Blues')
plt.title("Confusion Matrix: Titanic Survival Prediction")
plt.show()

# Step 14: SHAP Explainability
explainer = shap.DeepExplainer(model.cpu(), torch.tensor(X_train[:100], dtype=torch.float32))
shap_values = explainer.shap_values(torch.tensor(X_test[:10], dtype=torch.float32))
shap.summary_plot(shap_values[0], features=X_test[:10], feature_names=features)

# Step 15: Web Interface using Gradio
def predict_survival(pclass, sex, age, sibsp, parch, fare, embarked):
    # Convert inputs to the same scale and format used during training
    input_array = np.array([[pclass, sex, age, sibsp, parch, fare, embarked]])
    input_scaled = scaler.transform(input_array)
    input_tensor = torch.tensor(input_scaled, dtype=torch.float32).to(device)
    with torch.no_grad():
        output = model(input_tensor)
        prob = float(output.item())
        return {"Survived": round(prob, 3), "Did Not Survive": round(1 - prob, 3)}

# Define the Gradio interface with appropriate input types
iface = gr.Interface(
    fn=predict_survival,
    inputs=[
        gr.Slider(1, 3, step=1, label="Pclass (1 = 1st, 3 = 3rd)"),
        gr.Radio([0, 1], label="Sex (0 = Female, 1 = Male)"),
        gr.Number(label="Age"),
        gr.Number(label="Siblings/Spouses aboard"),
        gr.Number(label="Parents/Children aboard"),
        gr.Number(label="Fare"),
        gr.Radio([0, 1, 2], label="Embarked (0 = C, 1 = Q, 2 = S)")
    ],
    outputs="label",
    title="Titanic Survival Predictor"
)

# Launch the web interface
iface.launch()